## Geopandas

In [1]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
import geopandas as gpd # 지리정보에 특화된 데이터 구조와 기능을 제공하는 라이브러리
import pandas as pd
from shapely.geometry import Point # 지리 데이터를 기하학적 객체타입으로 표현하는 라이브러리/ 점(point)를 사용

# 압축 해제 후 .shp 파일 경로 지정
dong_gdf = gpd.read_file("C:\\Users\\choi heung ki\\BND_ADM_DONG_PG.shp")

# 좌표계 확인
print(dong_gdf.crs) # EPSG:5186 -> 투영좌표계

# 좌표계를 WGS84로 변환
dong_gdf = dong_gdf.to_crs(epsg=4326) # GPS 시스템 표준, 위/경도로 좌표값 나타냄
# Folium, plotly, Mapbox와 같은 웹기반 지도는 대부분 EPSG:4326형식을 사용

df_seoul = pd.read_csv("서울_충전소_좌표추가_전체_cleaned.csv")

# 위도, 경도를 이용해 Point 객체 생성
df_seoul['geometry'] = df_seoul.apply(lambda row: Point(row['경도'], row['위도']), axis=1) # 위/경도 데이터를 넣어 shapely의 point 객체를 생성

# GeoDataFrame으로 변환 (WGS84 사용)
# GeoDataFrame에는 geometry(지리적 객체)값을 담은 열이 꼭 하나 존재해야 한다.
gdf_seoul = gpd.GeoDataFrame(df_seoul, geometry='geometry', crs='EPSG:4326') 

# 공간 조인: 충전소가 포함된 행정동 정보를 병합
# predicate='within'으로 point 객체가 폴리곤 경계 내부에 위치하는지 GEOS 라이브러리를 통해 확인
gdf_joined = gpd.sjoin(gdf_seoul, dong_gdf[['ADM_CD', 'ADM_NM', 'geometry']], how='left', predicate='within')

# 결과 확인
print(gdf_joined[['충전소명', '주소', 'ADM_CD', 'ADM_NM']].head())

gdf_joined.to_csv("서울_충전소_행정동포함_new.csv", index=False, encoding='utf-8-sig')

EPSG:5186
          충전소명                    주소    ADM_CD       ADM_NM
0     낙성대동주민센터   서울특별시 관악구 낙성대로4가길 5  11240780         잠실7동
1     롯데마트 중계점     서울특별시 노원구 노원로 330  11220670         양재2동
2       서울추모공원  서울특별시 서초구 양재대로12길 74  11220670         양재2동
3  아시아공원 공영주차장      서울특별시 송파구 잠실동 84  11240780         잠실7동
4  아시아공원 공영주차장      서울특별시 송파구 잠실동 84  11010610  종로1·2·3·4가동


## 전체 데이터(수도권 + 제주도)에 행정동 코드/이름 추가

In [12]:
df = pd.read_csv("전국_충전소_좌표통합.csv")

df['geometry'] = df.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
gdf_all = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# 공간 조인: 충전소가 포함된 행정동 정보를 병합
gdf_joined = gpd.sjoin(gdf_all, dong_gdf[['ADM_CD', 'ADM_NM', 'geometry']], how='left', predicate='within')

# 결과 확인
print(gdf_joined[['충전소명', '주소', 'ADM_CD', 'ADM_NM']].head())

gdf_joined.to_csv("전국_충전소_행정동포함_new.csv", index=False, encoding='utf-8-sig')

          충전소명                    주소    ADM_CD       ADM_NM
0     낙성대동주민센터   서울특별시 관악구 낙성대로4가길 5  11240780         잠실7동
1     롯데마트 중계점     서울특별시 노원구 노원로 330  11220670         양재2동
2       서울추모공원  서울특별시 서초구 양재대로12길 74  11220670         양재2동
3  아시아공원 공영주차장      서울특별시 송파구 잠실동 84  11240780         잠실7동
4  아시아공원 공영주차장      서울특별시 송파구 잠실동 84  11010610  종로1·2·3·4가동


## 행정동별 충전소/충전기 갯수 파악

In [26]:
df = pd.read_csv("전국_충전소_행정동포함_new.csv")
df = df[['ADM_CD', 'ADM_NM', '충전소명', '충전기ID']]
df = df.dropna(subset=['ADM_CD', 'ADM_NM', '충전소명', '충전기ID']) # 결측값 제거
df = df.drop_duplicates() # 중복 제거

df_charger = df.groupby(['ADM_CD', 'ADM_NM'])['충전소명'].nunique().reset_index()
df_charger.columns = ['ADM_CD', 'ADM_NM', '충전소수']

df_connector = df.groupby(['ADM_CD', 'ADM_NM'])['충전기ID'].count().reset_index()
df_connector.columns = ['ADM_CD', 'ADM_NM', '충전기수']

df_result = pd.merge(df_charger, df_connector, on='ADM_CD')
df_result.to_csv("행정동별_충전소_충전기_집계_new.csv", index=False, encoding='utf-8-sig')

## 지역 컬럼 추가

In [32]:
df = pd.read_csv("행정동별_충전소_충전기_집계_new.csv", encoding="utf-8-sig")
# ADM_CD 문자열로 변환
df['ADM_CD'] = df['ADM_CD'].astype(str)

# 변환된 ADM_CD를 활용하여 지역(서울/경기/인천/제주) 구분
def classify_region(adm_cd):
    if adm_cd.startswith("11"):
        return "서울"
    elif adm_cd.startswith("31"):
        return "경기"
    elif adm_cd.startswith("23"):
        return "인천"
    elif adm_cd.startswith("39"):
        return "제주"
    else:
        return "기타"  # 예: 32 (강원), 29 (부산), 등등

# .apply()와 classify_region을 활용하여 지역 컬럼 추가
df['지역'] = df['ADM_CD'].apply(classify_region)

# ADM_CD가 32(강원)인 행 제거
# ~(부정, not)을 사용해 adm_cd시작이 32가 true면 false로, false면 true로 바꿔서 df에 저장
# 결과적으로 32(강원)이 아닌 행만 df에 저장
df = df[~df['ADM_CD'].str.startswith("32")] 
df.to_csv("행정동별_충전소_충전기_집계_지역포함_new.csv", index=False, encoding="utf-8-sig")


### 행정동 중심좌표 추가

In [136]:
gdf_adm_dong = gpd.read_file("C:\\Users\\choi heung ki\\BND_ADM_DONG_PG.shp")
print("로드된 shp파일 컬럼: ", gdf_adm_dong.columns)
# shp파일의 ADM_CD컬럼을 str타입으로 변경
gdf_adm_dong['ADM_CD'] = gdf_adm_dong['ADM_CD'].astype(str)

accurate_centroids = gdf_adm_dong.copy()
# centroid 메서드를 이용해 geometry에 있는 polygon 객체의 중심좌표 계산 후 x, y(경도, 위도)값 추출
# 투영좌표계로 먼저 centroid계산을 해야 더 정확하므로, 계산 후 지리좌표계로 변경
accurate_centroids['POINT'] = accurate_centroids.geometry.centroid
accurate_centroids['POINT'] = accurate_centroids['POINT'].to_crs(epsg=4326)
accurate_centroids['위도'] = accurate_centroids['POINT'].y
accurate_centroids['경도'] = accurate_centroids['POINT'].x
accurate_centroids = accurate_centroids[['ADM_CD', '위도', '경도']]

df = pd.read_csv("행정동별_충전소_충전기_집계_지역포함_new.csv", encoding='utf-8-sig')
df['ADM_CD'] = df['ADM_CD'].astype(str)

df_result = pd.merge(df, accurate_centroids, on='ADM_CD', how='left')
df_result.to_csv("행정동별_충전소_충전기_집계_중심좌표포함_new.csv", index=False, encoding='utf-8-sig')

로드된 shp파일 컬럼:  Index(['BASE_DATE', 'ADM_CD', 'ADM_NM', 'geometry'], dtype='object')


## 지도 시각화

### Plotley + Mapbox 활용 지도 시각화

In [1]:
pip install plotly pandas

In [148]:
import plotly.express as px # 고수준 Plotly API 모듈
import plotly.io as pio # Plotly로 생성한 그래프, 각종 형식으로 저장하거나 표시하는데 관련된 기능을 제공하는 모듈

# 데이터 불러오기
df = pd.read_csv("행정동별_충전소_충전기_집계_중심좌표포함_new.csv", encoding="utf-8-sig")

pio.renderers.default = 'browser'  # 또는 'browser' / 'iframe' 등 환경에 맞게
# 아래 YOUR_TOKEN_HERE 자리에 mapbox에서 받은 토큰을 입력
px.set_mapbox_access_token("YOUR_API_TOKEN")

fig = px.scatter_mapbox( # px.scatter_mapbox로 각종 지도 데이터가 담긴 fig 객체 생성
    df,
    lat="위도",
    lon="경도",
    size="충전기수",        # 점의 크기
    color="지역",           # 색상 분류
    hover_name="ADM_NM_x",  # 툴팁에 뜨는 이름
    hover_data=["충전소수", "충전기수"],
    size_max=25,
    zoom=7,
    title="행정동 중심 기반 전기차 충전기 밀도"
)

fig.update_layout(mapbox_style="light")  # 또는 "open-street-map", "dark", "light"
fig.show()